In [2]:
import pandas as pd

In [22]:
popo = pd.read_csv('./Data/경찰청_전국 치안센터 주소_20210203.csv', encoding = 'cp949')

In [24]:
popo = popo.loc[popo['지방청'].str.contains('서울')]

In [26]:
import json
with open('./Application_info.json', 'rb') as f:
     info = json.load(f)

In [27]:
def get_request_query(url, params, serviceKey):
    params = urlparse.urlencode(params)
    request_query = url + '?' + params + '&' + 'key' + '=' + serviceKey
    return request_query

In [28]:
def get_response_content(response):
    
    condition = response.ok
    
    df_temp = pd.DataFrame()
    
    if condition == True:
        content = response.content
        tmp = xmltodict.parse(content)['response']['result']['items']['item']
        df_temp = pd.DataFrame(tmp)
    else:
        print(f"status code : {response.status_code}")

    return df_temp

In [29]:
def collect_data_from_api(URL, SERVICEKEY, PARAMS):
    request_query = get_request_query(URL, PARAMS, SERVICEKEY)
    
    response = requests.get(url=request_query)

    root = get_response_content(response)
    
    return root

In [30]:
import urllib.request
import urllib.parse as urlparse
import requests
import xmltodict
import json

In [31]:
popo = popo.reset_index(drop = True)

In [34]:
url = 'http://api.vworld.kr/req/address'
params = {
    'service' : 'address',
    'request' : 'getcoord',
    'version' : '2.0',
    'crs' : 'epsg:4326',
    'address' : popo.loc[0,'주소'],
    'refine' : 'true',
    'simple' : 'false',
    'format' : 'xml',
    'type' : 'road'
}

key = info['Keys']['VWorld']

lons, lats = [], []

for i in range(len(popo)):
    if i % 10 == 0:
        print(i)
    params.update({'address': popo.loc[i,'주소']})
    
    request_query = get_request_query(url, params, key)
    request = requests.get(url=request_query)
    
    
    content = xmltodict.parse(request.content)
    
    if content['response']['status'] == 'NOT_FOUND':
        lon = 'NOT_FOUND'
        lat = 'NOT_FOUND'
    else:
        lon = content['response']['result']['point']['x']
        lat = content['response']['result']['point']['y']
        
    lons.append(lon)
    lats.append(lat)

print('Done')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160


In [42]:
lats[8] = '37.5501909'
lons[8] = '126.9721088'

lats[24] = '37.5790983'
lons[24] = '127.0522181'

lats[26] = '37.567768'
lons[26] = '127.0730669'

lats[34] = '37.5573016'
lons[34] = '126.9282706'

lats[52] = '37.496812'
lons[52] = '126.9819984'

lats[55] = '37.4988699'
lons[55] = '126.9254854'

lats[66] = '37.6631326'
lons[66] = '127.0127171'

lats[67] = '37.6443303'
lons[67] = '127.0152805'

lats[90] = '37.4818815'
lons[90] = '126.929143'

lats[91] = '37.479479'
lons[91] = '126.9264494'

#청룡2치안센터는 지도상 나타나지 않음

lats[99] = '37.5463112'
lons[99] = '127.1236774'

lats[100] = '37.5379319'
lons[100] = '127.1269891'

lats[101] = '37.5379319'
lons[101] = '127.1269891'

lats[103] = '37.5451403'
lons[103] = '127.1362782'

lats[105] = '37.5498247'
lons[105] = '127.1451476'

lats[108] = '37.6193705'
lons[108] = '127.0448482'

lats[112] = '37.5041037'
lons[112] = '126.8806198'

lats[127] = '37.5324456'
lons[127] = '126.8680492'

In [43]:
popo['lon'] = lons
popo['lat'] = lats

In [46]:
popo.loc[popo['lon'] == 'NOT_FOUND']

,연번,지방청,경찰서,관서명,지역파출소,치안센터명,주소,lon,lat
92,93,서울청,서울관악,신림,지구대,청룡2치안센터,서울 관악구 참숯1길 3,NOT_FOUND,NOT_FOUND


In [47]:
popo = popo.drop(popo.loc[popo['lon'] == 'NOT_FOUND'].index)

In [50]:
popo.to_csv('./Data/서울시_치안센터_지오코딩.csv', index=False)